In [3]:
from pyscf import gto,scf
import pyscf
import matplotlib.pyplot as plt
import basis_set_exchange as bse
from scipy.interpolate import InterpolatedUnivariateSpline as spl
import numpy as np
from FcMole import FcM
ang2bohr=1.8897261246
bohr2ang=.5291772109

In [14]:
CO=gto.M(atom="C 0 0 0; O 0 0 2.05",unit='Bohrs',
                      basis=bse.get_basis("pcX-2",fmt="nwchem",elements=[5,6,7,8,9]))
mf=scf.RHF(CO)
CO.nao

90

In [4]:
def optimizer2at(a1,a2,fcs=None):
    d1=np.linspace(1.9,2.4,9)
    es=[]
    for d in d1:
        if fcs==None:
            mol=gto.M(atom="{} 0 0 0; {} 0 0 {} ".format(a1,a2,d),unit='Bohrs',
                      basis=bse.get_basis("pcX-2",fmt="nwchem",elements=[5,6,7,8,9]),verbose=0)
            mf=scf.RHF(mol)
            es.append(mf.scf())
        else:
            mol=FcM(fcs=fcs,atom="{} 0 0 0; {} 0 0 {} ".format(a1,a2,d),unit='Bohrs',
                      basis=bse.get_basis("pcX-2",fmt="nwchem",elements=[5,6,7,8,9]),verbose=0)
            mf=scf.RHF(mol)
            es.append(mf.scf(dm0=mf.init_guess_by_1e()))
    dspl=np.linspace(1.9,2.4,501)
    bspline=spl(d1,es)
    bspline(dspl)
    dminspline=1.9+.001*(bspline(dspl).argmin())
    d2=np.linspace(dminspline-.03,dminspline+.03,7)
    es=[]
    for d in d2:
        if fcs==None:
            mol=gto.M(atom="{} 0 0 0; {} 0 0 {} ".format(a1,a2,d),unit='Bohrs',
                      basis=bse.get_basis("pcX-2",fmt="nwchem",elements=[5,6,7,8,9]),verbose=0)
            mf=scf.RHF(mol)
            es.append(mf.scf())
        else:
            mol=FcM(fcs=fcs,atom="{} 0 0 0; {} 0 0 {} ".format(a1,a2,d),unit='Bohrs',
                      basis=bse.get_basis("pcX-2",fmt="nwchem",elements=[5,6,7,8,9]),verbose=0)
            mf=scf.RHF(mol)
            es.append(mf.scf(dm0=mf.init_guess_by_1e()))
    dspl=np.linspace(dminspline-.03,dminspline+.03,6001)
    bspline=spl(d2,es)
    #plt.plot(dspl,bspline(dspl))
    COmin=dspl[bspline(dspl).argmin()]
    return COmin,bspline(dspl).min()

In [5]:
optimizer2at("C","O")

(2.0827199999999997, -112.78661621869807)

In [6]:
optimizer2at("N","N")

(2.0138899999999995, -108.98906407528155)

In [7]:
optimizer2at("B","F")

(2.35348, -124.16243198073053)

In [8]:
#CO@NN
optimizer2at("N","N",fcs=[-1,1.])

(2.08391, -112.78107642188813)

In [9]:
#CO@BF
optimizer2at("B","F",fcs=[1.,-1.])

(2.0832399999999995, -112.78531725440988)

In [10]:
#BF@CO
optimizer2at("C","O",fcs=[-1,1])

(2.35483, -124.15657992324608)

In [11]:
#NN@CO
optimizer2at("C","O",fcs=[1,-1])

(2.0138199999999995, -108.9890173918064)

In [18]:
import pandas as pd

In [19]:
q=[]
l={"basis set":"pcX-2"
    ,"nAO":90
    ,"COm":2.0827199999999997
    ,"COe":-112.78661621869807
    ,"NNm":2.0138899999999995
    ,"NNe":-108.98906407528155
    ,"BFm":2.35348
    ,"BFe":-124.16243198073053
    ,"CO@NNm":2.08391
    ,"CO@NNe": -112.78107642188813
    ,"CO@BFm":2.0832399999999995
    ,"CO@BFe":-112.7853172544098
    ,"BF@COm":2.35483
    ,"BF@COe":-124.15657992324608
    ,"NN@COm":2.0138199999999995
    ,"NN@COe":-108.9890173918064
}
q.append(l)

In [21]:
df=pd.DataFrame(q)
df.to_pickle("pcX2")